<center>
<a target="_blank" href="http://eeit.sec.gob.mx/">
  <img src="http://eeit.sec.gob.mx/assets/media/others/bubble-12.png" width=250pt style="padding-bottom:5px;" />
</a>
<br/><br/><br/>
<a target="_blank" href="https://educacion.sonora.gob.mx/">
  <img src="https://educacion.sonora.gob.mx/images/2023/07/17/logo-sec.svg" width=150pt style="padding-bottom:5px;" />
</a>
</center>

<h1><b>Transferencia de aprendizaje en reconocimiento de imágenes</b> </h1>

<author>Julio Waissman Vilanova</author>

<br/>

<a target="_blank" href="https://colab.research.google.com/github/juliowaissman/eeit2024/blob/main/transfer.ipynb">
<img src="https://i.ibb.co/2P3SLwK/colab.png" width=30pt />
<i>Para usar en Google Colab</i></a>

## Introducción

En esta libreta vamos a presentar la aplicación de transferencia de aprendizaje para clasificación de imágenes.
Esta es la base para buscar otros modelos y otras aplicaciones posibles.

In [1]:
import itertools
import os

# Las que no pueden faltar
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf

# Para obtener los modelos preentrenados
import tensorflow_hub as hub

Esta libreta muestra cómo reutilizar un modelo existente  usando como ejemplo la clasificación de cinco especies de flores. Se va a utilizar un modelo entrenado en el conjunto de datos ImageNet y guardado en [TensorFlow Hub](https://www.tensorflow.org/hub).

Se van a mantener las capas de extracción de características y unicamente se va a modificar el clasificador.

Opcionalmente, si se tiene tiempo y capacidad de cómputo, esposible usar un ajuste fino de los pesos del extractor de características.

Este ejemplo está tomado de [este ejemplo de la documentación de tensorflow](https://www.tensorflow.org/hub/tutorials/tf2_image_retraining).

## Obteniendo las imágenes del conjunto de entrenamiento y prueba

Vamos a descargar las imágenes y guardarlas en un directorio dentro de nuestro espacio de trabajo (al finalizar la sesión se borrarán si se ejecuta en *Colab*).

In [ ]:
data_dir = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True
)

¿Y que tiene lo que acabamos de descargar y donde lo descargamos? Veamos:

In [ ]:
!ls -l ~/.keras/datasets/flower_photos/

## Selección de un modelo preentrenado

Vamos ahora a seleccionar un modelo a descargar. Para esto vamos a poner una lista (tomada obviamente del tutorial desarrollado por el equipo de TensorFlow Hub) de los modelos disponibles, así como el tamaño de las imágenes que usa cada uno de los modelos. Los vamos a guardar en un diccionario:

In [ ]:
model_handle_map = {
  "efficientnetv2-s": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/feature_vector/2",
  "efficientnetv2-m": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_m/feature_vector/2",
  "efficientnetv2-l": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_l/feature_vector/2",
  "efficientnetv2-s-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2",
  "efficientnetv2-m-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/feature_vector/2",
  "efficientnetv2-l-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_l/feature_vector/2",
  "efficientnetv2-xl-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/feature_vector/2",
  "efficientnetv2-b0-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/feature_vector/2",
  "efficientnetv2-b1-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b1/feature_vector/2",
  "efficientnetv2-b2-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b2/feature_vector/2",
  "efficientnetv2-b3-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b3/feature_vector/2",
  "efficientnetv2-s-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_s/feature_vector/2",
  "efficientnetv2-m-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_m/feature_vector/2",
  "efficientnetv2-l-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_l/feature_vector/2",
  "efficientnetv2-xl-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2",
  "efficientnetv2-b0-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/feature_vector/2",
  "efficientnetv2-b1-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b1/feature_vector/2",
  "efficientnetv2-b2-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b2/feature_vector/2",
  "efficientnetv2-b3-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2",
  "efficientnetv2-b0": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2",
  "efficientnetv2-b1": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b1/feature_vector/2",
  "efficientnetv2-b2": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b2/feature_vector/2",
  "efficientnetv2-b3": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b3/feature_vector/2",
  "efficientnet_b0": "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
  "efficientnet_b1": "https://tfhub.dev/tensorflow/efficientnet/b1/feature-vector/1",
  "efficientnet_b2": "https://tfhub.dev/tensorflow/efficientnet/b2/feature-vector/1",
  "efficientnet_b3": "https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1",
  "efficientnet_b4": "https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1",
  "efficientnet_b5": "https://tfhub.dev/tensorflow/efficientnet/b5/feature-vector/1",
  "efficientnet_b6": "https://tfhub.dev/tensorflow/efficientnet/b6/feature-vector/1",
  "efficientnet_b7": "https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1",
  "bit_s-r50x1": "https://tfhub.dev/google/bit/s-r50x1/1",
  "inception_v3": "https://tfhub.dev/google/imagenet/inception_v3/feature-vector/4",
  "inception_resnet_v2": "https://tfhub.dev/google/imagenet/inception_resnet_v2/feature-vector/4",
  "resnet_v1_50": "https://tfhub.dev/google/imagenet/resnet_v1_50/feature-vector/4",
  "resnet_v1_101": "https://tfhub.dev/google/imagenet/resnet_v1_101/feature-vector/4",
  "resnet_v1_152": "https://tfhub.dev/google/imagenet/resnet_v1_152/feature-vector/4",
  "resnet_v2_50": "https://tfhub.dev/google/imagenet/resnet_v2_50/feature-vector/4",
  "resnet_v2_101": "https://tfhub.dev/google/imagenet/resnet_v2_101/feature-vector/4",
  "resnet_v2_152": "https://tfhub.dev/google/imagenet/resnet_v2_152/feature-vector/4",
  "nasnet_large": "https://tfhub.dev/google/imagenet/nasnet_large/feature_vector/4",
  "nasnet_mobile": "https://tfhub.dev/google/imagenet/nasnet_mobile/feature_vector/4",
  "pnasnet_large": "https://tfhub.dev/google/imagenet/pnasnet_large/feature_vector/4",
  "mobilenet_v2_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
  "mobilenet_v2_130_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/4",
  "mobilenet_v2_140_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4",
  "mobilenet_v3_small_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5",
  "mobilenet_v3_small_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5",
  "mobilenet_v3_large_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5",
  "mobilenet_v3_large_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5",
}

model_image_size_map = {
  "efficientnetv2-s": 384,
  "efficientnetv2-m": 480,
  "efficientnetv2-l": 480,
  "efficientnetv2-b0": 224,
  "efficientnetv2-b1": 240,
  "efficientnetv2-b2": 260,
  "efficientnetv2-b3": 300,
  "efficientnetv2-s-21k": 384,
  "efficientnetv2-m-21k": 480,
  "efficientnetv2-l-21k": 480,
  "efficientnetv2-xl-21k": 512,
  "efficientnetv2-b0-21k": 224,
  "efficientnetv2-b1-21k": 240,
  "efficientnetv2-b2-21k": 260,
  "efficientnetv2-b3-21k": 300,
  "efficientnetv2-s-21k-ft1k": 384,
  "efficientnetv2-m-21k-ft1k": 480,
  "efficientnetv2-l-21k-ft1k": 480,
  "efficientnetv2-xl-21k-ft1k": 512,
  "efficientnetv2-b0-21k-ft1k": 224,
  "efficientnetv2-b1-21k-ft1k": 240,
  "efficientnetv2-b2-21k-ft1k": 260,
  "efficientnetv2-b3-21k-ft1k": 300,
  "efficientnet_b0": 224,
  "efficientnet_b1": 240,
  "efficientnet_b2": 260,
  "efficientnet_b3": 300,
  "efficientnet_b4": 380,
  "efficientnet_b5": 456,
  "efficientnet_b6": 528,
  "efficientnet_b7": 600,
  "inception_v3": 299,
  "inception_resnet_v2": 299,
  "nasnet_large": 331,
  "pnasnet_large": 331,
}

print('Los modelos que podemos escoger son los siguientes:')

print('\n'.join(model_handle_map.keys()))

Y ahora si, seleccionamos nuestro modelo, el número de pixeles que maneja de imágenes de entrada y el tamaño del minibatch para reentrenar

In [ ]:
model_name = "efficientnetv2-xl-21k"

model_handle = model_handle_map.get(model_name)
pixels = model_image_size_map.get(model_name, 224)

print(f"Modelo seleccionado: {model_name} : {model_handle}")

IMAGE_SIZE = (pixels, pixels)
print(f"Tamaño de la imagen: {IMAGE_SIZE}")

BATCH_SIZE = 16

## Flujo de trabajo y preprocesamiento de datos

Ahora vamos a procesar los datos, y para eso vamos a usar el [modulo ``tf.data`](https://www.tensorflow.org/guide/data?hl=en) que permite crear *pipelines* para consumir (potencialmente) grandes cantidades de datos.

Vamos a seleccionar un conjunto de entrenamiento con el 80% de los datos. Se separan los valores de salida y se generan minibatches de tamaño `BATCH_SIZE``.

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.20,
  subset="training",
  label_mode="categorical",
  seed=123, # Para asgurar la reproducibilidad de los conjuntos de datos
  image_size=IMAGE_SIZE,
  batch_size=1 # Un dato por batch solo para sacar el tamaño de la muestra
)
train_size = train_ds.cardinality().numpy()

class_names = tuple(train_ds.class_names)

# Ahora si, hacenmos batches de tamaño BATCH_SIZE
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()

Y ahora el conjunto de validación

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  label_mode="categorical",
  seed=123,
  image_size=IMAGE_SIZE,
  batch_size=1
)
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)

Como unico paso de preprocesamiento, vamos a reescalar (normalizar) el valor de las imágenes. Recuerda que los valores de los canales RGB de cada pixel están dados desde 0 hasta 255 (8 bits, $2^8 = 256 valores diferentes).

In [9]:
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])

train_ds = train_ds.map(
    lambda images, labels: (preprocessing_model(images), labels)
)
val_ds = val_ds.map(
    lambda images, labels: (normalization_layer(images), labels)
)

## Diseño del modelo de aprendizaje

Vamos a rediseñar el modelo de aprendizaje a partir del modelo descargado. Lo que vamos a hacer es sustituir la última capa del modelo. Para un modelo entrenado con el conjunto de datos de [ImageNet](https://www.image-net.org/) es una capa tipo *softmax* con 1,000 valores diferentes (clasifica entre 1,000 categorías). Esta la vamos a sustituir por un problema mñas modesto de aprender entre 5 clases de flores diferentes.

In [ ]:
do_fine_tuning = False

print("Modelo basado en ", model_handle)
print("¿Vamos a hacer ajuste fino? ", do_fine_tuning)

model = tf.keras.Sequential([
  # Define explicitamente el input shape
  tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),

  # Carga el modelo preentrenado, con o sin fine tunning
  hub.KerasLayer(model_handle, trainable=do_fine_tuning),

  # Dropout para ayudar a la generalización
  tf.keras.layers.Dropout(rate=0.2),

  # Capa de salida lineal del número de clases y con regularización
  tf.keras.layers.Dense(
      len(class_names),
      kernel_regularizer=tf.keras.regularizers.l2(0.0001)
  )
  ],
  name='Mi-modelo-bien-bonito'
)

# Construye el modelo haciendo explicito el tamaño de entrada
model.build((None,) + IMAGE_SIZE + (3,))

model.summary()

## Compilar, entrenar y validar

Ahora compilamos y decidimos que optimizador, función de pérdida y métricas utilizar.

In [11]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy']
)

In [ ]:
EPOCHS = 3
steps_per_epoch = train_size // BATCH_SIZE
validation_steps = valid_size // BATCH_SIZE

hist = model.fit(
    train_ds,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps
).history

Y ahora grafiquemos la función de pérdida y la métrica seleccionada por los *epochs* que escogimos simular.

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1,2,1)
plt.plot(hist["loss"], label='Entrenamiento')
plt.plot(hist["val_loss"], label='Validación')
plt.ylabel("Funcion de pérdida")
plt.xlabel("Pasos de aprendizaje")

plt.subplot(1,2,2)
plt.plot(hist["accuracy"], label='Entrenamiento')
plt.plot(hist["val_accuracy"], label='Validación')
plt.ylabel("% de error")
plt.xlabel("Pasos de aprendizaje")

plt.show()

## Uso del modelo reentrenado

Probemos como se enviaría una imagen nueva al clasificador:

In [ ]:
# Con el iterador se obtiene un minibatch de imágenes y clases
x, y = next(iter(val_ds))

# Tomemos la primer imagen y su indice
image = x[0, :, :, :]
true_index = np.argmax(y[0])

# Mostramos la imagen
plt.imshow(image)
plt.axis('off')
plt.show()

# Expande la imagen a (1, 224, 224, 3) para introducirla al modelo
prediction_scores = model.predict(np.expand_dims(image, axis=0))

# Obten el indice con mayor valor en la capa de salida
predicted_index = np.argmax(prediction_scores)

print("True label: " + class_names[true_index])
print("Predicted label: " + class_names[predicted_index])

Y si el modelo te gustó y lo quieres usar más adelate, sólamente hay que guardarlo.

In [18]:
saved_model_path = f"/tmp/saved_flowers_model_{model_name}"
tf.saved_model.save(model, saved_model_path)

In [ ]:
!ls -l /tmp